# Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')
from rdkit.Chem import MACCSkeys

[18:29:29] Initializing Normalizer


# Data entry and curation work set

In [3]:
uploaded_file_ws="datasets/KRAS_work_from_insilico.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  452 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [6]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [8]:
len(y_tr)

452

# Standardization SDF file for work set

In [11]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  452 molecules


In [12]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
1,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
2,<rdkit.Chem.rdchem.Mol object at 0x000001D4E1F...
3,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
4,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
...,...
447,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
448,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
449,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
450,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...


# Data entry and curation test set

In [16]:
uploaded_file_ts="datasets/KRAS_test_from_insilico.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  114 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [19]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [21]:
len(y_ts)

114

# Standardization SDF file for test set

In [24]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  114 molecules


In [25]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
1,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
2,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
3,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
4,<rdkit.Chem.rdchem.Mol object at 0x000001D4E2C...
...,...
109,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
110,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
111,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...
112,<rdkit.Chem.rdchem.Mol object at 0x000001D4E3C...


# Calculation MorganFingerprint for work set

In [29]:
def calcfp(mol):
    fp = MACCSkeys.GenMACCSKeys(mol)
    fp = pd.Series(np.asarray(fp))
    fp = fp.add_prefix('MACCS_')
    return fp

# Training set
desc_ws = moldf_ws.Mol.apply(calcfp)
desc_ws

,MACCS_0,MACCS_1,MACCS_2,MACCS_3,MACCS_4,MACCS_5,MACCS_6,MACCS_7,MACCS_8,MACCS_9,...,MACCS_157,MACCS_158,MACCS_159,MACCS_160,MACCS_161,MACCS_162,MACCS_163,MACCS_164,MACCS_165,MACCS_166
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
448,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
449,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
450,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0


In [30]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

452

# Calculation MorganFingerprint for test set

In [34]:
desc_ts = moldf_ts.Mol.apply(calcfp)
desc_ts

,MACCS_0,MACCS_1,MACCS_2,MACCS_3,MACCS_4,MACCS_5,MACCS_6,MACCS_7,MACCS_8,MACCS_9,...,MACCS_157,MACCS_158,MACCS_159,MACCS_160,MACCS_161,MACCS_162,MACCS_163,MACCS_164,MACCS_165,MACCS_166
0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
110,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
111,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
112,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [35]:
y_ts = np.array(y_ts, dtype=np.float32)

# BASELINE

 ## GradientBoostingRegressor model building and validation

In [40]:
seed = 42

In [42]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [44]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(desc_ws, y_tr, verbose=False)

CPU times: total: 32.9 s
Wall time: 6min 29s


,estimator,<catboost.cor...001D4E5072C30>
,param_grid,"{'depth': [6, 8, ...], 'iterations': [100, 500, ...], 'learning_rate': [0.01, 0.05, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False


In [45]:
best_CatBR = grid.best_estimator_

In [46]:
grid.best_params_

{'depth': 6, 'iterations': 1000, 'learning_rate': 0.01}

In [47]:
params={'verbose': False}

In [48]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, desc_ws, y_tr, cv=cv, params=params)

CPU times: total: 1min 46s
Wall time: 11.3 s


In [49]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.48

In [50]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.88

In [107]:
y_pred_GBR = best_CatBR.predict(desc_ts)

In [109]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.46

In [111]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.91

save the model to disk

In [57]:
pickle.dump(best_CatBR, open('Models/MACCS/CatBoost_MACCS.pkl', 'wb'))

load the model from disk

In [67]:
best_CatBR = pickle.load(open('Models/MACCS/CatBoost_MACCS.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [114]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [116]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,1.732051,1.414214,1.000000,0.000000,2.000000,1.000000,2.645751,0.000000,2.645751,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.414214
2,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.645751,1.732051,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000
3,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.236068,0.000000,0.000000,2.000000
4,2.000000,2.000000,2.000000,1.732051,1.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.236068,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.196152,5.291503,5.385165,5.291503,5.385165,5.099020,5.567764,6.000000,6.480741,6.000000,...,5.196152,5.385165,5.291503,5.291503,5.196152,5.385165,5.567764,5.385165,5.385165,5.477226
448,5.196152,5.291503,5.477226,5.291503,5.477226,5.099020,5.567764,6.000000,6.480741,6.082763,...,5.291503,5.477226,5.385165,5.291503,5.291503,5.477226,5.567764,5.477226,5.477226,5.567764
449,5.196152,5.291503,5.477226,5.291503,5.477226,5.291503,5.830952,6.082763,6.782330,6.082763,...,5.291503,5.477226,5.385165,5.385165,5.291503,5.477226,5.656854,5.477226,5.477226,5.567764
450,5.291503,5.385165,5.477226,5.477226,5.656854,5.291503,5.830952,6.164414,6.782330,6.164414,...,5.477226,5.656854,5.567764,5.477226,5.477226,5.656854,5.656854,5.656854,5.656854,5.656854


In [118]:
similarity= neighbors_k

In [120]:
Dmean=np.mean(similarity[1,:])

In [122]:
round(Dmean, 2)

0.78

In [124]:
std=np.std(similarity[1,:])

In [126]:
round(std, 2)

0.87

In [128]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.22


In [130]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=desc_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [132]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.000000,1.732051,0.000000,0.000000,1.732051,1.414214,0.000000,1.414214,0.000000,0.000000,...,1.000000,1.732051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.828427,2.000000,0.000000,0.000000,2.000000,1.732051,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.414214,0.000000,0.000000
2,2.828427,2.236068,0.000000,0.000000,2.000000,2.000000,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.236068,0.000000,0.000000,0.000000,1.000000,0.000000,1.732051,1.000000,0.000000
3,2.828427,2.236068,0.000000,0.000000,2.000000,2.236068,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,0.000000,1.000000,0.000000,1.000000,0.000000,2.449490,1.000000,0.000000
4,2.828427,2.449490,0.000000,0.000000,2.000000,2.449490,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,1.000000,1.000000,0.000000,1.000000,0.000000,2.645751,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.567764,5.291503,5.477226,5.291503,5.291503,5.477226,5.291503,5.385165,5.567764,5.291503,...,5.567764,5.656854,5.385165,5.291503,5.291503,5.291503,5.291503,5.656854,5.196152,5.385165
448,5.656854,5.567764,5.477226,5.291503,5.291503,5.567764,5.385165,5.385165,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.291503,5.385165,5.656854,5.385165,5.477226
449,5.656854,5.656854,5.477226,5.291503,5.477226,5.656854,5.385165,5.477226,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.385165,5.385165,5.744563,5.385165,5.477226
450,5.830952,5.656854,5.656854,5.567764,5.477226,5.656854,5.567764,5.477226,5.656854,5.567764,...,5.830952,5.830952,5.656854,5.477226,5.567764,5.477226,5.567764,5.744563,5.477226,5.656854


In [134]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    1.732 0.    0.    1.732 1.414 0.    1.414 0.    0.    1.    3.162
 1.    0.    1.414 1.732 2.    2.    0.    0.    1.732 1.414 1.732 1.732
 0.    1.414 1.414 1.    1.    1.    0.    1.414 0.    2.449 1.414 0.
 0.    1.732 2.    0.    1.    0.    1.    0.    1.    1.    0.    0.
 0.    1.    0.    0.    1.    1.414 1.414 2.    0.    0.    2.    1.732
 0.    2.449 0.    0.    0.    0.    0.    2.    1.732 1.    1.    0.
 0.    0.    1.    0.    2.    2.236 1.    0.    1.    2.    0.    1.414
 0.    0.    0.    0.    3.    0.    0.    0.    1.    1.414 1.    0.
 1.732 0.    0.    0.    0.    1.414 1.    1.732 1.    1.732 0.    0.
 0.    0.    0.    0.    0.    0.   ]


In [136]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False  True  True False False  True False  True  True  True False
  True  True False False False False  True  True False False False False
  True False False  True  True  True  True False  True False False  True
  True False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False  True  True False False
  True False  True  True  True  True  True False False  True  True  True
  True  True  True  True False False  True  True  True False  True False
  True  True  True  True False  True  True  True  True False  True  True
 False  True  True  True  True False  True False  True False  True  True
  True  True  True  True  True  True]


In [138]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.6666666666666666


In [86]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   2   3   6   8   9  10  12  13  18  19  24  27  28  29  30  32  35
  36  39  40  41  42  43  44  45  46  47  48  49  50  51  52  56  57  60
  62  63  64  65  66  69  70  71  72  73  74  75  78  79  80  82  84  85
  86  87  89  90  91  92  94  95  97  98  99 100 102 104 106 107 108 109
 110 111 112 113]


In [88]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [141]:
y_pred_GBR_ad=list(y_pred_GBR)

In [143]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [145]:
len(y_pred_GBR_ad)

76

In [147]:
y_ts_ad=list(y_ts)

In [149]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [151]:
len(y_ts_ad)

76

In [153]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.37

In [155]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.97

# SVM model building and validation

In [158]:
from sklearn.svm import SVR

In [160]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [162]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [164]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [166]:
svm.fit(desc_ws, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


,estimator,SVR(epsilon=0.2)
,param_grid,"{'C': [1, 10, ...], 'gamma': [1e-06, 1e-05, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,kernel,'rbf'


In [167]:
svm.best_params_
best_svm = svm.best_estimator_

In [168]:
svm.best_params_

{'C': 1, 'gamma': 0.1}

In [169]:
y_pred_CV_svm = cross_val_predict(best_svm, desc_ws, y_tr, cv=cv)

In [170]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.48

In [171]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.88

#  Prediction for test set's molecules

In [173]:
x_ts = np.array(desc_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [174]:
y_pred_svm = best_svm.predict(x_ts)

In [183]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.46

In [184]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.91

save the model to disk

In [188]:
pickle.dump(best_svm, open('Models/MACCS/MACCS_FP.pkl', 'wb'))

load the model from disk

In [114]:
best_svm = pickle.load(open('Models/MACCS/MACCS_FP.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [190]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [192]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,1.732051,1.414214,1.000000,0.000000,2.000000,1.000000,2.645751,0.000000,2.645751,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.414214
2,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.645751,1.732051,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000
3,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.236068,0.000000,0.000000,2.000000
4,2.000000,2.000000,2.000000,1.732051,1.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.236068,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.196152,5.291503,5.385165,5.291503,5.385165,5.099020,5.567764,6.000000,6.480741,6.000000,...,5.196152,5.385165,5.291503,5.291503,5.196152,5.385165,5.567764,5.385165,5.385165,5.477226
448,5.196152,5.291503,5.477226,5.291503,5.477226,5.099020,5.567764,6.000000,6.480741,6.082763,...,5.291503,5.477226,5.385165,5.291503,5.291503,5.477226,5.567764,5.477226,5.477226,5.567764
449,5.196152,5.291503,5.477226,5.291503,5.477226,5.291503,5.830952,6.082763,6.782330,6.082763,...,5.291503,5.477226,5.385165,5.385165,5.291503,5.477226,5.656854,5.477226,5.477226,5.567764
450,5.291503,5.385165,5.477226,5.477226,5.656854,5.291503,5.830952,6.164414,6.782330,6.164414,...,5.477226,5.656854,5.567764,5.477226,5.477226,5.656854,5.656854,5.656854,5.656854,5.656854


In [194]:
similarity= neighbors_k

In [196]:
Dmean=np.mean(similarity[1,:])

In [198]:
round(Dmean, 2)

0.78

In [200]:
std=np.std(similarity[1,:])

In [202]:
round(std, 2)

0.87

In [204]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.22


In [206]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [208]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.000000,1.732051,0.000000,0.000000,1.732051,1.414214,0.000000,1.414214,0.000000,0.000000,...,1.000000,1.732051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.828427,2.000000,0.000000,0.000000,2.000000,1.732051,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.414214,0.000000,0.000000
2,2.828427,2.236068,0.000000,0.000000,2.000000,2.000000,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.236068,0.000000,0.000000,0.000000,1.000000,0.000000,1.732051,1.000000,0.000000
3,2.828427,2.236068,0.000000,0.000000,2.000000,2.236068,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,0.000000,1.000000,0.000000,1.000000,0.000000,2.449490,1.000000,0.000000
4,2.828427,2.449490,0.000000,0.000000,2.000000,2.449490,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,1.000000,1.000000,0.000000,1.000000,0.000000,2.645751,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.567764,5.291503,5.477226,5.291503,5.291503,5.477226,5.291503,5.385165,5.567764,5.291503,...,5.567764,5.656854,5.385165,5.291503,5.291503,5.291503,5.291503,5.656854,5.196152,5.385165
448,5.656854,5.567764,5.477226,5.291503,5.291503,5.567764,5.385165,5.385165,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.291503,5.385165,5.656854,5.385165,5.477226
449,5.656854,5.656854,5.477226,5.291503,5.477226,5.656854,5.385165,5.477226,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.385165,5.385165,5.744563,5.385165,5.477226
450,5.830952,5.656854,5.656854,5.567764,5.477226,5.656854,5.567764,5.477226,5.656854,5.567764,...,5.830952,5.830952,5.656854,5.477226,5.567764,5.477226,5.567764,5.744563,5.477226,5.656854


In [210]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    1.732 0.    0.    1.732 1.414 0.    1.414 0.    0.    1.    3.162
 1.    0.    1.414 1.732 2.    2.    0.    0.    1.732 1.414 1.732 1.732
 0.    1.414 1.414 1.    1.    1.    0.    1.414 0.    2.449 1.414 0.
 0.    1.732 2.    0.    1.    0.    1.    0.    1.    1.    0.    0.
 0.    1.    0.    0.    1.    1.414 1.414 2.    0.    0.    2.    1.732
 0.    2.449 0.    0.    0.    0.    0.    2.    1.732 1.    1.    0.
 0.    0.    1.    0.    2.    2.236 1.    0.    1.    2.    0.    1.414
 0.    0.    0.    0.    3.    0.    0.    0.    1.    1.414 1.    0.
 1.732 0.    0.    0.    0.    1.414 1.    1.732 1.    1.732 0.    0.
 0.    0.    0.    0.    0.    0.   ]


In [212]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False  True  True False False  True False  True  True  True False
  True  True False False False False  True  True False False False False
  True False False  True  True  True  True False  True False False  True
  True False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False  True  True False False
  True False  True  True  True  True  True False False  True  True  True
  True  True  True  True False False  True  True  True False  True False
  True  True  True  True False  True  True  True  True False  True  True
 False  True  True  True  True False  True False  True False  True  True
  True  True  True  True  True  True]


In [214]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [216]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   2   3   6   8   9  10  12  13  18  19  24  27  28  29  30  32  35
  36  39  40  41  42  43  44  45  46  47  48  49  50  51  52  56  57  60
  62  63  64  65  66  69  70  71  72  73  74  75  78  79  80  82  84  85
  86  87  89  90  91  92  94  95  97  98  99 100 102 104 106 107 108 109
 110 111 112 113]


In [218]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [221]:
y_pred_svm_ad=list(y_pred_svm)

In [223]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [225]:
len(y_pred_svm_ad)

76

In [227]:
y_ts_ad=list(y_ts)

In [229]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [231]:
len(y_ts_ad)

76

In [233]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.37

In [235]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.98

# Multi-layer Perceptron regressor

In [238]:
from sklearn.neural_network import MLPRegressor

In [240]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [242]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [244]:
m.fit(desc_ws, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


,estimator,MLPRegressor()
,param_grid,"{'activation': ['tanh', 'relu'], 'alpha': [5e-05, 0.0005], 'hidden_layer_sizes': [(400, ...), (100, ...), ...], 'max_iter': [1000, 2000], ...}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'squared_error'


In [245]:
best_MLPR = m.best_estimator_

In [246]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, desc_ws, y_tr, cv=cv)

In [247]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.44

In [248]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.91

# Prediction for test set's molecules

In [256]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [258]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.44

In [260]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.93

save the model to disk

In [262]:
pickle.dump(best_MLPR, open('models/MACCS/MLPR_MACCS.pkl', 'wb'))

#  Estimating applicability domain. Method - Euclidian distances, K=1

In [265]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [267]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.000000,1.732051,1.414214,1.000000,0.000000,2.000000,1.000000,2.645751,0.000000,2.645751,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.414214
2,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.645751,1.732051,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000
3,2.000000,2.000000,2.000000,1.414214,0.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.236068,0.000000,0.000000,2.000000
4,2.000000,2.000000,2.000000,1.732051,1.000000,2.000000,1.000000,2.828427,3.162278,2.828427,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.236068,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.196152,5.291503,5.385165,5.291503,5.385165,5.099020,5.567764,6.000000,6.480741,6.000000,...,5.196152,5.385165,5.291503,5.291503,5.196152,5.385165,5.567764,5.385165,5.385165,5.477226
448,5.196152,5.291503,5.477226,5.291503,5.477226,5.099020,5.567764,6.000000,6.480741,6.082763,...,5.291503,5.477226,5.385165,5.291503,5.291503,5.477226,5.567764,5.477226,5.477226,5.567764
449,5.196152,5.291503,5.477226,5.291503,5.477226,5.291503,5.830952,6.082763,6.782330,6.082763,...,5.291503,5.477226,5.385165,5.385165,5.291503,5.477226,5.656854,5.477226,5.477226,5.567764
450,5.291503,5.385165,5.477226,5.477226,5.656854,5.291503,5.830952,6.164414,6.782330,6.164414,...,5.477226,5.656854,5.567764,5.477226,5.477226,5.656854,5.656854,5.656854,5.656854,5.656854


In [269]:
similarity= neighbors_k

In [271]:
Dmean=np.mean(similarity[1,:])

In [273]:
round(Dmean, 2)

0.78

In [275]:
std=np.std(similarity[1,:])

In [277]:
round(std, 2)

0.87

In [279]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.22


In [281]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [283]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.000000,1.732051,0.000000,0.000000,1.732051,1.414214,0.000000,1.414214,0.000000,0.000000,...,1.000000,1.732051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.828427,2.000000,0.000000,0.000000,2.000000,1.732051,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.414214,0.000000,0.000000
2,2.828427,2.236068,0.000000,0.000000,2.000000,2.000000,0.000000,2.236068,1.732051,0.000000,...,1.000000,2.236068,0.000000,0.000000,0.000000,1.000000,0.000000,1.732051,1.000000,0.000000
3,2.828427,2.236068,0.000000,0.000000,2.000000,2.236068,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,0.000000,1.000000,0.000000,1.000000,0.000000,2.449490,1.000000,0.000000
4,2.828427,2.449490,0.000000,0.000000,2.000000,2.449490,0.000000,2.449490,2.000000,0.000000,...,1.414214,2.236068,1.000000,1.000000,0.000000,1.000000,0.000000,2.645751,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,5.567764,5.291503,5.477226,5.291503,5.291503,5.477226,5.291503,5.385165,5.567764,5.291503,...,5.567764,5.656854,5.385165,5.291503,5.291503,5.291503,5.291503,5.656854,5.196152,5.385165
448,5.656854,5.567764,5.477226,5.291503,5.291503,5.567764,5.385165,5.385165,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.291503,5.385165,5.656854,5.385165,5.477226
449,5.656854,5.656854,5.477226,5.291503,5.477226,5.656854,5.385165,5.477226,5.567764,5.385165,...,5.744563,5.744563,5.477226,5.385165,5.385165,5.385165,5.385165,5.744563,5.385165,5.477226
450,5.830952,5.656854,5.656854,5.567764,5.477226,5.656854,5.567764,5.477226,5.656854,5.567764,...,5.830952,5.830952,5.656854,5.477226,5.567764,5.477226,5.567764,5.744563,5.477226,5.656854


In [285]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    1.732 0.    0.    1.732 1.414 0.    1.414 0.    0.    1.    3.162
 1.    0.    1.414 1.732 2.    2.    0.    0.    1.732 1.414 1.732 1.732
 0.    1.414 1.414 1.    1.    1.    0.    1.414 0.    2.449 1.414 0.
 0.    1.732 2.    0.    1.    0.    1.    0.    1.    1.    0.    0.
 0.    1.    0.    0.    1.    1.414 1.414 2.    0.    0.    2.    1.732
 0.    2.449 0.    0.    0.    0.    0.    2.    1.732 1.    1.    0.
 0.    0.    1.    0.    2.    2.236 1.    0.    1.    2.    0.    1.414
 0.    0.    0.    0.    3.    0.    0.    0.    1.    1.414 1.    0.
 1.732 0.    0.    0.    0.    1.414 1.    1.732 1.    1.732 0.    0.
 0.    0.    0.    0.    0.    0.   ]


In [287]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False  True  True False False  True False  True  True  True False
  True  True False False False False  True  True False False False False
  True False False  True  True  True  True False  True False False  True
  True False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False  True  True False False
  True False  True  True  True  True  True False False  True  True  True
  True  True  True  True False False  True  True  True False  True False
  True  True  True  True False  True  True  True  True False  True  True
 False  True  True  True  True False  True False  True False  True  True
  True  True  True  True  True  True]


In [289]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.67


In [291]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   2   3   6   8   9  10  12  13  18  19  24  27  28  29  30  32  35
  36  39  40  41  42  43  44  45  46  47  48  49  50  51  52  56  57  60
  62  63  64  65  66  69  70  71  72  73  74  75  78  79  80  82  84  85
  86  87  89  90  91  92  94  95  97  98  99 100 102 104 106 107 108 109
 110 111 112 113]


In [293]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [296]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [298]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [300]:
len(y_pred_MLPR_ad)

76

In [302]:
y_ts_ad=list(y_ts)

In [304]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [306]:
len(y_ts_ad)

76

In [308]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.36

In [310]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.99